In [1]:
load('Imports/Utility.sage', 'Imports/Point.sage', 'Imports/Line.sage', 'Imports/TritangentPlane.sage', 'Imports/Group.sage', 'Imports/Cubic.sage')

In [2]:
P.<x,y,z,t,b,c,d,e,f,l,m> = PolynomialRing(QQ)
cubic_new = e*f*(2*x^2*y-2*x*y^2+x*z^2-x*z*t-y*t^2+y*z*t)+b*c*(x-t)*(x*z+y*t)+c*c*(z+t)*(y*t-x*z)+d*c*(y-z)*(x*z+y*t)+(e+f)*c*(x-y)*(y*t-x*z)
sing_cubics = (-1) * (-c + f) * (-c + e) * c * (c + f) * (c + e) * (-e + f)^2 * (-c*d + c*f + e*f) * (-c*d + c*e + e*f) * (-c^2 - c*d + e*f) * (b*c - c*f + e*f) * (b*c - c*e + e*f) * (b*c - c*d + 2*e*f) * (b*c - c^2 + e*f) * (b*c^2 + c^2*d + b*c*f - 2*c^2*f - c*d*f + 2*e*f^2) * (b*c^2 + c^2*d + b*c*e - 2*c^2*e - c*d*e + 2*e^2*f) * (-b*c^3 - 2*b*c^2*d + c^3*d + b*c^2*e + c^2*d*e + b*c^2*f + c^2*d*f + 3*b*c*e*f - 4*c^2*e*f - 3*c*d*e*f + 4*e^2*f^2)
line = Line([y, z])
general_cubic = Cubic(cubic_new, line, sing_cubics)

In [3]:
L_set_base = general_cubic.get_L_set_in_plucker(['E1', 'G4', 'E2', 'G3', 'E3'])

In [4]:
# First two are from when a line is fixed, other three when no line fixed. Excluded identity
possible_L_sets = [ ('E1', 'G4', 'E2', 'G3', 'F45'),\
 ('E1', 'G4', 'E2', 'G3', 'F46'),\
 ('F14', 'E1', 'G3', 'F23', 'G5'),\
 ('F14', 'E1', 'G3', 'F23', 'G6'),\
 ('F14', 'E1', 'G3', 'F23', 'F12')]

In [5]:
%%time 
# this takes approximately 20 seconds 
projs = [find_projectivity(L_set_base, general_cubic.get_L_set_in_plucker(L_set_2)) for L_set_2 in possible_L_sets]
projs_ideals = []
coeffs = []
sing_cubics_factored = sing_cubics.factor()
for proj in projs:
    #change coordinates, obtain new cubic and find conditions on the parameters
    sost = change_coord(proj)
    new_cubic = general_cubic.eqn.subs(sost)
    mon = (sum(P.gens()[0:4]) ^ 3).monomials()
    current_conds = list(set(matrix([[general_cubic.eqn.coefficient(mn) for mn in mon], [new_cubic.coefficient(mn) for mn in mon]]).minors(2)))
    no_sing_conds = [remove_sing_factors(el, sing_cubics_factored) for el in current_conds if el !=0]
    
    #two branches: assume e*f is zero and add to groebner basis, otherwise saturate with respect to e*f
    ideals = []
    gb = [remove_sing_factors(el, sing_cubics_factored) for el in P.ideal(no_sing_conds).groebner_basis()] 
    ideals += P.ideal(gb+[e*f]).radical().primary_decomposition('gtz')
    
    idealb = P.ideal(gb).saturation(P.ideal(f*e))[0]
    ideals += idealb.radical().primary_decomposition('gtz')
    
    #remove duplicates and singular ideals
    ideals = list(set(ideals))
    ideals = [ideal for ideal in ideals if sing_cubics not in ideal]
    
    # we obtain a list where, for each projectivity, there are the possible associated nonsingular ideals
    projs_ideals.append([proj, ideals])

CPU times: user 11.7 s, sys: 59.4 ms, total: 11.8 s
Wall time: 11.9 s


In [6]:
#number of ideals for each projectivity
[len(el[1]) for el in projs_ideals]

[1, 1, 2, 2, 3]

In [7]:
# here we obtain the matrices with the sostitutions obtained from the various ideals
matrices_from_ideals = []
all_matrices = []
for matrix_ideals in projs_ideals:
    proj = matrix_ideals[0]
    ideals = matrix_ideals[1]
    
    # matrices_from_ideals remembers from which ideals the matrices where generated
    matrices_from_ideals += [[matrix(4,4, [ideal.reduce(el) for el in proj.list()]), ideal] for ideal in ideals]

In [8]:
matrices_from_ideals_cubed = [[el[0]^3, el[1]] for el in matrices_from_ideals]
identity_list = [0 for _ in range(16)]
for i in {0,5,10,15}:
    identity_list[i]=1
id_mat = matrix(4,4,identity_list)

In [80]:
final_matrices = []
final_ideals = []
for mat_id in matrices_from_ideals_cubed:
    final_matrices.append(matrix(4,4, [mat_id[1].reduce(el) for el in mat_id[0].list()]))
    final_ideals.append(mat_id[1])

In [79]:
for mat_id in matrices_from_ideals_cubed:
    print(are_matrices_equal(matrix(4,4,[mat_id[1].reduce(el) for el in mat_id[0].list()]), id_mat))

True
True
True
True
True
True
True
True
True


In [82]:
cubics = [general_cubic.reduce(ideal) for ideal in final_ideals]

In [83]:
[len(cubic.eckardt_points_labels) for cubic in cubics]

[18, 18, 9, 9, 9, 9, 3, 3, 3]

In [53]:
ide = final_ideals[0]
ide

Ideal (e + f, 4*c - d, b + d, 3*d^2 + 16*f^2) of Multivariate Polynomial Ring in x, y, z, t, b, c, d, e, f, l, m over Rational Field

In [16]:
(general_cubic.cl_lines['F15'].intersection_point(general_cubic.cl_lines['F26'])).reduce(ide)

(4*f, 4*f, 3*d, 3*d)

In [17]:
general_cubic.cl_lines['F15'].reduce(ide).intersection_point(general_cubic.cl_lines['F26'].reduce(ide))

ValueError: Lines are not incident!

In [56]:
general_cubic.cl_lines['F15'].plucker, general_cubic.cl_lines['F26'].plucker

((b*c^2 - c^3 + c*e*f, b*c*e - c^2*e + e^2*f, c^2*d - c*d*e - c^2*f + e^2*f, 0, -b*c^2 + c^2*e - c^2*f, -b*c*e + c*e^2 - c*e*f),
 (-c^3 - c^2*d + c*e*f, -c^2*d - c^2*e + c^2*f, 0, b*c^2 - c^2*e + b*c*f + e*f^2, -c^2*f - c*d*f + e*f^2, -c*d*f - c*e*f + c*f^2))

In [69]:
general_cubic.cl_lines['F15'].are_incident(general_cubic.cl_lines['F26'])

True

In [60]:
Point([ide.reduce(el) for el in general_cubic.cl_lines['F15'].intersection_point( general_cubic.cl_lines['F26'])])

(4*f, 4*f, 3*d, 3*d)

In [68]:
general_cubic.cl_lines['F15'].reduce(ide).plucker, general_cubic.cl_lines['F26'].reduce(ide).plucker

((d, -4*f, -2*d, 0, -2*d + 4*f, 3*d + 8*f),
 (d, 2*d - 4*f, 0, 2*d, 4*f, 3*d + 8*f))

In [71]:
r1 = general_cubic.cl_lines['F15'].reduce(ide)
r2 = general_cubic.cl_lines['F26'].reduce(ide)

In [75]:
d = [r1.plucker[i] * r2.plucker[5 - i] for i in range(6)]
poly = d[0] - d[1] + d[2] + d[3] - d[4] + d[5]
ide.reduce(poly)

0

In [76]:
ide

Ideal (e + f, 4*c - d, b + d, 3*d^2 + 16*f^2) of Multivariate Polynomial Ring in x, y, z, t, b, c, d, e, f, l, m over Rational Field